## Build ground truth tuple from ath dataset

In [8]:
import pickle
import numpy as np

with open("nt_seq_dic.pkl", 'rb') as p:
    nt_seq_dic = pickle.load(p)
with open("gene_subtree_dic.pkl", 'rb') as p:
    gene_subtree_dic = pickle.load(p)
with open("total_BR_tree_dic.pkl", 'rb') as p:
    total_BR_tree_dic = pickle.load(p)

len(gene_subtree_dic.keys())

63

In [9]:
from mytree import *
import copy

count=0
gene_node_list = []
current_name = "Photosynthesis proteins"

for i in gene_subtree_dic.keys():
    count+=1
    tmp_node, _ = find_deepest_node(gene_subtree_dic[i], current_name)
    gene_node_list.append(tmp_node)


root = total_BR_tree_dic['ko00194']
print(cal_tree_distance(root, gene_node_list[-4], gene_node_list[-5]))

# clean up gene names
nice_name_gene_node_list = copy.deepcopy(gene_node_list)
for i in range(len(gene_node_list)):
    if i < 8:
        nice_name_gene_node_list[i].value = gene_node_list[i].value[16:].split(';')[0]
    else:
        nice_name_gene_node_list[i].value = gene_node_list[i].value[8:].split(';')[0]

4


In [10]:
flag={}
notin = set()
for i in nt_seq_dic.keys():
    for j in nice_name_gene_node_list:
        if i.lower() in j.value.lower():
            flag[i] = flag.get(i, 0)+1
            break
        elif j == nice_name_gene_node_list[-1]:
            notin.add(i)

In [11]:
correct = {
    'CYTC6A': 'petJ',
    'PSAE-2': 'psaE',
    'PSBP-1': 'psbP',
    'PnsL2': 'psbQ',
    'NPQ4': 'psbS',
    'PSAD-2': 'psaD',
    'PSAH2': 'psaH',
    'DRT112': 'petE',
    'FD1': 'petF',
    'FNR2':'petH'
}

In [12]:
# clean names in nice_name_gene_node_lise
for k,v in correct.items():
    for i in nice_name_gene_node_list:
        if v == i.value:
            i.value = k

In [13]:
notin

{'ATPC2',
 'CYTC6A',
 'DRT112',
 'FD1',
 'FNR2',
 'NPQ4',
 'PSAD-2',
 'PSAE-2',
 'PSAH2',
 'PSBO2',
 'PSBP-1',
 'PnsL2',
 'atpI'}

In [14]:
fixed_gene_names = list(nt_seq_dic.keys())
fixed_gene_names.remove('atpI')
fixed_gene_names.remove('PSBO2')
fixed_gene_names.remove('ATPC2')
only_names_gene_node = []

length = 0
for i in fixed_gene_names:
    for j in nice_name_gene_node_list:
        if i.lower() == j.value.lower():
            length+=1

for i in nice_name_gene_node_list:
    only_names_gene_node.append(i.value.lower())

print(length)

ground_truth_tree_dist = np.zeros((length, length), dtype=float)
for i in range(length):
    index = only_names_gene_node.index(fixed_gene_names[i].lower())
    node_i = gene_node_list[i]
    for j in range(length):
        index = only_names_gene_node.index(fixed_gene_names[j].lower())
        node_j = gene_node_list[j]
        ground_truth_tree_dist[i, j] = cal_tree_distance(root, node_i, node_j)

51


In [145]:
with open('ground_truth_dist_tuple.pkl', 'wb') as p:
    pickle.dump((fixed_gene_names, ground_truth_tree_dist), p)

## Build ground truth tuple from human dataset

In [ ]:
from mytree import *
from get_gene_name_and_seq import get_NT_seq
import numpy as np
import pickle
import copy
from tqdm import tqdm

with open('human_total_BR_tree_dic.pkl', 'rb') as p:
    human_dic = pickle.load(p)

root = human_dic['sa00001']
deepest_nodes_list, _ = find_deepest_node(root, '', from_root_get_all=True)

index = list(range(len(deepest_nodes_list)))
human_gene_name_list = ["hsa:"+deepest_nodes_list[i].value.split()[0] for i in index]
human_gene_ntseq_link_list = ["https://www.kegg.jp/entry/"+i for i in human_gene_name_list]

# Test: select random 100 genes
np.random.seed(0)
sample_size = 10
index = np.random.choice(len(deepest_nodes_list), sample_size, replace=False)
human_gene_name_list = ["hsa:"+deepest_nodes_list[i].value.split()[0] for i in index]
human_gene_ntseq_link_list = ["https://www.kegg.jp/entry/"+i for i in human_gene_name_list]

real_name_list = []
# get all nt seq and build a dictionary
human_nt_seq_dic = {}
for i in tqdm(range(len(human_gene_name_list))):
    name, seq = get_NT_seq(human_gene_ntseq_link_list[i])
    real_name_list.append(name)
    human_nt_seq_dic[name] = seq

# clculate the distance matrix
length = len(real_name_list)
ground_truth_tree_dist = np.zeros((length, length), dtype=float)
for i in index:
    node_i = deepest_nodes_list[i]
    for j in index:
        node_j = deepest_nodes_list[j]
        ground_truth_tree_dist[i, j] = cal_tree_distance(root, node_i, node_j)

---

In [139]:
with open('human_enzymes_BR_tree_dic.pkl', 'rb') as p:
    human_enzymes_dic = pickle.load(p)

root = human_enzymes_dic['sa01000']
node1 = root.children[0].children[0].children[0].children[2]
node2 = root.children[0].children[0].children[0].children[2]

print(cal_tree_distance(root, node1, node2))

0


In [135]:
def cal_tree_distance(root, node1, node2):
    lca = find_lca(root, node1, node2)
    distance1 = find_distance_to_ancestor(lca, node1)
    distance2 = find_distance_to_ancestor(lca, node2)
    return distance1 + distance2
def find_lca(root, node1, node2):
    if root is None:
        return None
    if not root.value is None:
        if root.value.lower() == node1.value.lower() or root.value.lower() == node2.value.lower():
            return root

    lca_node = None
    node1_found = False
    node2_found = False

    for child in root.children:
        lca = find_lca(child, node1, node2)
        if lca:
            if lca.value.lower() == node1.value.lower():
                node1_found = True
            if lca.value.lower() == node2.value.lower():
                node2_found = True
            if not lca_node:
                lca_node = lca
            if node1_found and node2_found:
                return lca_node

    if node1_found and node2_found:
        return root
    return lca_node

In [8]:
# clculate the distance matrix
root = human_dic['sa00001']
length = len(real_name_list)
ground_truth_tree_dist = np.zeros((length, length), dtype=float)
for i in index:
    node_i = deepest_nodes_list[i]
    for j in index:
        node_j = deepest_nodes_list[j]
        ground_truth_tree_dist[i, j] = mydis(root, node_i, node_j)

NameError: name 'human_dic' is not defined